Now that we've gotten our hands dirty with Python and some of its important operations, it's time to take on Tensorflow. Tensorflow is Google's language platform for deep learning, and it is one - if not the - most important platforms for deep learning available to the public. It can be used on its own or through higher-level frameworks such as Caffe and Keras, which can be set to use Tensorflow as a backend. 

*Note: consider adding for visual clarity https://blog.jakuba.net/2017/05/30/tensorflow-visualization.html*

### Foreword - Take a Deep Breath..

Fear not, but Tensorflow is extraordinarily complex and can be quite complicated - almost too so. It has layers upon layers of abstractions and a horde of APIs which seem to gain and fall from prominence each day. Backwards compatibility is often an issue. Such is the life of a rapidly developin ecosystem I suppose. 

Because of the above, we will come across very many methods of writing Tensorflow programs as we explore some of the most interesting models currently available. It is COMPLETELY NORMAL to feel overwhelmed and confused at first. Do not worry about this. Our method in this course is not to try and understand everything at once, but focus on areas of interest to understand how they work and how we may work with them. We will expand our knowledge in this fashion, by combining what we learn. In time, the whole picture of Tensorflow and machine/deep learning will become clearer.

### High-Level / Low-Level

One can create the same model in 30 lines of code or in hundreds of lines. We are going to do both. Why? Well, the 30-line model is so *high-level* that we won't really understand what those 30 lines of code are doing, for everything is hidden in high-level abstractions. So we can look at lower-level code, which will have many more lines but show us more clearly what exactly is happening. Once we understand how the model works on a lower-level, we can work more knowledgeably and quickly with higher-level programming again.


### Tensorflow and Python - BFFs Forever?

Tensorflow works hand-in-hand with the Python framework and many of its methods are basically exact replicas of Python/Numpy code. This is generally a good thing because it gives us familiarity as we work between the two languages. The reason for this partnership is that their similarities and compatibility support a fundamental difference in execution: **immediate/eager evaluation** vs. **graph computation**

In short, Python will evaluate (run) the code as soon as it sees it. Tensorflow, on the other hand, does not immediately run its operations. Instead, it builds a so-called *graph*, which can be considered like a system of how each variable and operation are connected to one another. In order to execute (run) the graph, we must explicitly tell Tensorflow to do so. This will become apparent in the example code below.

Final note - this introductory code has been taken from several sources, who often do a better job explaining then I can, so we will follow along with some external sites as we go. To start with, please open [this tutorial](https://jacobbuckman.com/post/tensorflow-the-confusing-parts-1/#understanding-tensorflow) to read along with the example code below.

In [2]:
import tensorflow as tf

# create a tf.Variable
one_node = tf.Variable(2)

#create a Python variable
py_var = 2

print("tf graph element: %s" % (one_node,),"python variable execution: %s" % (py_var,))

tf graph element: <tf.Variable 'Variable_1:0' shape=() dtype=int32_ref> python variable execution: 2


Note that the Tensorflow code above did not evaluate, but returned instead a graph element called a **tensor** (more on that later). On the other hand, the Python variable was immediately evaluated and returned 2.

This creates some very important differences. For example, in Python if I assign 2 to the same variable, we still have 1 variable with a value of 2. If I assign this variable a value of 3, then we have the same 1 variable but with a value of 3. In Tensorflow, the graph does something quite different. Let's see for ourselves below.

In [3]:
two_node = tf.constant(2)
print(two_node)

Tensor("Const:0", shape=(), dtype=int32)


In [4]:
two_node = tf.constant(2)
two_node = tf.constant(2)
two_node = tf.constant(2)
tf.constant(3)

<tf.Tensor 'Const_4:0' shape=() dtype=int32>

Notice that in the first code block we have 'Const' and then in the second code block we have 'Const_4'. This means each time we call tf.constant it adds a tf.constant to the graph regardless of where we assign the variable in our code.

If we don't wish for this to happen, then we should create "pointer" variables instead of calling new instances of tf variables.

In [5]:
pointer_at_two_node = two_node
print(pointer_at_two_node)

Tensor("Const_3:0", shape=(), dtype=int32)


Above, we can notice 2 things. First, we didn't create another tf.constant but simply pointed the variable already storing it. Also, the tf.constant is 'Const_3' instead of 'Const_4' because we created 'Const_4' without assigning it to a Python variable. The variable we are "pointing" to contains 'Const_3'

So what we have created so far are called **nodes** in tensorflow - that is, individual values. When we want to compute them together somehow, this is called an **operation**.

In [6]:
two_node = tf.constant(2)
three_node = tf.constant(3)
sum_node = two_node + three_node ## equivalent to tf.add(two_node, three_node)
print(sum_node)

Tensor("add:0", shape=(), dtype=int32)


Though very simple, we have our 1st **computational graph**. Notice that it has not been evaluated yet, for when we print sum_node we get the operation "add", not 5 like we would if this were just in Python.

In order to evaluate a graph, we need to begin what is called a **session**. This means the graph is interactive and can be executed (evaluated)

In [7]:
sess = tf.Session()

print(sess.run(sum_node))

5


Perhaps we want the values of nodes as well as operations

In [8]:
print(sess.run([two_node, sum_node]))

[2, 5]


And we can return them to Python variables if we want

In [9]:
node_val, op_result = sess.run([two_node, sum_node])
print(node_val, op_result, node_val+op_result)

2 5 7


Sometimes when creating a graph, we want to add in values at a later point (for instance, perhaps we need to insert an image file into our graph and evaluate it). To do so, we can make use of **tf.placeholder** and **feed_dict**. As the name suggests, placeholder simply holds an empty location in the graph. It has no value until we pass in a value using feed_dict. 

Below we'll feed a tf.placeholder with a Python variable of value 2.

In [10]:
input_placeholder = tf.placeholder(tf.int32)
feed_value = 2
print(sess.run(input_placeholder, feed_dict={input_placeholder: feed_value}))

2


So far we've evaluated graphs that contain static or "no-ancestor" nodes - they don't update. However, with machine/deep learning, **we often need nodes that update** - this is how they learn! Most of the **parameters** we want to train/learn will be implemented as Tensorflow **variables**. 

Also, we've only been dealing with **singular values**, called **scalars** so far. Generally, we will be working with **matrices** of varying dimensions. To do this, we assign variables a **shape**, which is their dimensions. 

Finally, we can also give the variable a **name** to make it easier to identify, or Tensorflow will do it for us (e.g. 'const_5', etc.)

In [11]:
count_variable = tf.get_variable("count", [2, 3])
zero_node = tf.constant([[1, 2, 3], [4, 5, 6]])
assign_node = tf.assign(count_variable, zero_node)
sess = tf.Session()
sess.run(assign_node)
print(sess.run(count_variable))

TypeError: Input 'value' of 'Assign' Op has type int32 that does not match type float32 of argument 'ref'.

Note we received an error exception for data type incompatibility. We often need to declare which data type we want to work with to avoid this error

In [12]:
count_variable = tf.get_variable("count", [2, 3], dtype=tf.float32)
zero_node = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
assign_node = tf.assign(count_variable, zero_node)
sess = tf.Session()
sess.run(assign_node)
print(sess.run(count_variable))

ValueError: Variable count already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-11-5dd14a5ec98a>", line 1, in <module>
    count_variable = tf.get_variable("count", [2, 3])
  File "/Users/jonathansherman/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/jonathansherman/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


Oh, now another error exception - this time for variable reuse. This is because we have given it a specific name, so Tensorflow cannot simply add another variable like it did when we allowed it to autoname variables. And because the error in our code was further below the count_variable, it thinks we are trying to add it again and it is blocking us. Simple workaround when this happens - comment out the count_variable until the block of code works, then include it next time

In [13]:
#count_variable = tf.get_variable("count", [2, 3], dtype=tf.float32)
zero_node = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
assign_node = tf.assign(count_variable, zero_node)
sess = tf.Session()
sess.run(assign_node)
print(sess.run(count_variable))

[[1. 2. 3.]
 [4. 5. 6.]]


Or reset the "default graph" which will clear variables and let us start anew.

In [16]:
tf.reset_default_graph()
count_variable = tf.get_variable("count", [2, 3], dtype=tf.float32)
zero_node = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
assign_node = tf.assign(count_variable, zero_node)
sess = tf.Session()
sess.run(assign_node)
print(sess.run(count_variable))

[[1. 2. 3.]
 [4. 5. 6.]]


Great, so it finally worked, but perhaps is still confusing. Why this "get_variable" and "assign" process? Why can't we just create a variable in one step. Well, we can with **tf.Variable()**, but the "pointer" method above makes sure we aren't creating unnecessary graph elements like we've shown before. For instance, if there already is a variable with the name we are looking for, we don't want to create another. tf.Variable() will always try to create another.

In [17]:
pointer_count_variable = tf.get_variable("count", [2, 3], dtype=tf.float32)

ValueError: Variable count already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-16-63a0e12acad5>", line 2, in <module>
    count_variable = tf.get_variable("count", [2, 3], dtype=tf.float32)
  File "/Users/jonathansherman/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/jonathansherman/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


In [18]:
create_additional_count_var = tf.Variable(1, name = 'count')
print(create_additional_count_var)

<tf.Variable 'count_1:0' shape=() dtype=int32_ref>


See above, get_variable was an error exception as we expected, whereas tf.Variable created a new node with a 0 value, autonamed 'count_1'. If this is not what we wanted, we will have problems with our graph.

Finally, there is another way to create and evaluate variables other than using the 'assign' method. We can use "initializers", which will set all variables or constants to an initial value. Since these will generally be updated by our training, we often don't care what the initial value is, or just set it to 0.

In [19]:
const_init_node = tf.constant_initializer(0.)
count_variable = tf.get_variable("count", [], initializer=const_init_node)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
print(sess.run(count_variable))

ValueError: Variable count already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-16-63a0e12acad5>", line 2, in <module>
    count_variable = tf.get_variable("count", [2, 3], dtype=tf.float32)
  File "/Users/jonathansherman/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/jonathansherman/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


Oh yeah, that annoying error again. We've "accidentally" created a bunch of nodes we don't want, and are trying to change an existing node. Let's reset all of the variables and get this right by using **tf.reset_default_graph**

In [28]:
tf.reset_default_graph()

const_init_node = tf.constant_initializer(0.)
count_variable = tf.get_variable("count", [], initializer=const_init_node)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
print(sess.run(count_variable))

0.0


Hopefully this gives you an idea of how Tensorflow works with but is different than Python, and how the concept of building a graph with nodes and operations differes and *then* evaluating it is different than the immediate evaluation we are probably used to with Python or other coding languages.

There is plenty more to learn about Tensorflow, but this is a good start. 

Let's see how this is put together in a toy **linear regression** example.

Most importantly, pay attention to how the variables are updated as the model trains and evaluates. This is called **optimization**, and a typical optimization will do something along the likes of the following:

1. Get an input and true_output
2. Compute a “guess” based on the input and your parameters
3. Compute a “loss” based on the difference between your guess and the true_output
4. Update the parameters according to the gradient of the loss

In [29]:
### build the graph
## first set up the parameters
m = tf.get_variable("m", [], initializer=tf.constant_initializer(0.))
b = tf.get_variable("b", [], initializer=tf.constant_initializer(0.))
init = tf.global_variables_initializer()

## then set up the computations
input_placeholder = tf.placeholder(tf.float32)
output_placeholder = tf.placeholder(tf.float32)

x = input_placeholder
y = output_placeholder
y_guess = m * x + b

loss = tf.square(y - y_guess)

## finally, set up the optimizer and minimization node
optimizer = tf.train.GradientDescentOptimizer(1e-3)
train_op = optimizer.minimize(loss)

### start the session
sess = tf.Session()
sess.run(init)

### perform the training loop
import random

## set up problem
true_m = random.random()
true_b = random.random()

for update_i in range(10000):
    ## (1) get the input and output
    input_data = random.random()
    output_data = true_m * input_data + true_b
    
    ## (2), (3), and (4) all take place within a single call to sess.run()!
    _loss, _ = sess.run([loss, train_op], feed_dict={input_placeholder: input_data, output_placeholder: output_data})
    print(update_i, _loss)
    

### finally, print out the values we learned for our two variables
print("True parameters:     m=%.4f, b=%.4f" % (true_m, true_b))
print("Learned parameters:  m=%.4f, b=%.4f" % tuple(sess.run([m, b])))

0 0.35693797
1 0.3260934
2 0.33125213
3 0.33489585
4 0.36332142
5 0.35614416
6 0.34536755
7 0.3352813
8 0.30892545
9 0.3449778
10 0.32310665
11 0.31132305
12 0.3165279
13 0.32155466
14 0.31069866
15 0.3424847
16 0.33180934
17 0.31547776
18 0.32336363
19 0.3366809
20 0.2968938
21 0.2951831
22 0.30387324
23 0.3160706
24 0.28880665
25 0.3240013
26 0.3222623
27 0.30477986
28 0.30631647
29 0.28929874
30 0.3039982
31 0.28207445
32 0.28969717
33 0.2795231
34 0.29336676
35 0.29042163
36 0.27643892
37 0.28648344
38 0.29885253
39 0.28172052
40 0.27252346
41 0.27737424
42 0.27948517
43 0.29042757
44 0.27758938
45 0.26723903
46 0.28299528
47 0.27699503
48 0.27262244
49 0.25991732
50 0.26984423
51 0.27669534
52 0.27342945
53 0.27671027
54 0.2622346
55 0.2698363
56 0.26677495
57 0.2543423
58 0.2518611
59 0.2621873
60 0.26453415
61 0.2631657
62 0.25380546
63 0.24984607
64 0.2553891
65 0.24509516
66 0.24753182
67 0.24946748
68 0.24410109
69 0.24075152
70 0.24068806
71 0.24037614
72 0.24043612
73 0.239

638 0.02204986
639 0.0146067245
640 0.010420342
641 0.03223714
642 0.0040197903
643 0.011357198
644 0.0033118462
645 0.01473932
646 0.014492912
647 0.023596523
648 0.010748161
649 0.006204466
650 0.003923674
651 0.016030721
652 0.021067763
653 0.003682399
654 0.007449349
655 0.007015961
656 0.0044951243
657 0.02864715
658 0.015766507
659 0.009468054
660 0.03106975
661 0.0065227076
662 0.014917209
663 0.018757576
664 0.016344601
665 0.0067959046
666 0.012224285
667 0.012442871
668 0.02735232
669 0.0034076755
670 0.0181745
671 0.005818702
672 0.016057773
673 0.029050725
674 0.006330701
675 0.019963093
676 0.014265548
677 0.022961529
678 0.022138216
679 0.003789638
680 0.00216634
681 0.025792591
682 0.016802438
683 0.00460795
684 0.010780039
685 0.02080355
686 0.025957925
687 0.0030382792
688 0.007693801
689 0.022076119
690 0.009006533
691 0.015245446
692 0.023456225
693 0.019233866
694 0.004706639
695 0.025305925
696 0.0029997064
697 0.008211257
698 0.008455
699 0.0043296493
700 0.020667

1244 0.008343514
1245 0.010236552
1246 0.002200066
1247 0.0005942056
1248 0.0008851505
1249 5.302456e-06
1250 0.00037908927
1251 0.009054247
1252 1.4423959e-05
1253 0.0016605452
1254 0.0012239839
1255 3.674605e-05
1256 0.0017598665
1257 0.009102793
1258 0.00086066243
1259 2.4402143e-06
1260 0.00010042106
1261 0.00058067235
1262 0.010880963
1263 0.0077448236
1264 0.00047190307
1265 0.008864606
1266 0.00053571403
1267 0.01121643
1268 9.521009e-06
1269 0.0049888515
1270 0.0029769698
1271 4.4888922e-05
1272 0.009336598
1273 0.00017733108
1274 0.0011231338
1275 0.00064132543
1276 0.0022877473
1277 0.0055584363
1278 0.010198793
1279 0.009293257
1280 0.0018931166
1281 0.0043083243
1282 0.0004472138
1283 0.00069574104
1284 0.007765457
1285 0.009520435
1286 0.0011637042
1287 0.0010873479
1288 0.00030623272
1289 0.0050643845
1290 0.0013090413
1291 7.761357e-06
1292 0.0011923157
1293 0.004543424
1294 0.0010140795
1295 0.0009017014
1296 0.005359146
1297 0.0010602074
1298 0.00017096015
1299 0.00798

1859 0.00020286834
1860 0.0005543571
1861 7.589883e-05
1862 8.779968e-05
1863 0.0013008119
1864 0.006415676
1865 0.0048866775
1866 0.0015319291
1867 0.0049594883
1868 0.0010720206
1869 1.761282e-05
1870 0.0033250311
1871 0.0015902891
1872 0.003959741
1873 0.0008837608
1874 0.0013639324
1875 0.004088467
1876 0.001781255
1877 0.0025128685
1878 0.0041994913
1879 0.0002216096
1880 5.1757674e-05
1881 0.0038428812
1882 3.7783742e-05
1883 3.8752387e-06
1884 0.0067463624
1885 0.0039633573
1886 0.005666407
1887 0.004993648
1888 7.6738215e-05
1889 0.0005495595
1890 5.437219e-05
1891 0.006815908
1892 0.003483098
1893 0.003584396
1894 0.0022114876
1895 0.00644587
1896 0.0043161367
1897 2.1796877e-05
1898 0.002332962
1899 0.0009360871
1900 0.0029634694
1901 0.0015050793
1902 0.0012731855
1903 0.004710062
1904 0.00015081858
1905 0.00016768252
1906 4.1468018e-05
1907 1.986057e-05
1908 0.0036010733
1909 0.006106324
1910 0.0007086006
1911 0.0055796267
1912 1.2415513e-06
1913 0.002937426
1914 0.00242489

2472 0.0007505976
2473 0.00046637753
2474 0.0032070193
2475 0.0030666788
2476 0.0023792523
2477 0.0012303483
2478 8.740916e-05
2479 0.00015928326
2480 0.0011064449
2481 4.6877656e-05
2482 0.00029602228
2483 5.9506306e-07
2484 0.0015469103
2485 0.0016777862
2486 0.0026735675
2487 1.7474508e-06
2488 0.0019154804
2489 0.000218633
2490 0.000687049
2491 0.0006546103
2492 0.003530806
2493 3.7102443e-05
2494 0.00069861795
2495 0.0026988992
2496 0.0022329937
2497 0.0031635368
2498 0.0038638602
2499 7.3054405e-05
2500 0.00037619352
2501 0.001601757
2502 0.0017597014
2503 0.0026243676
2504 0.001304228
2505 0.002197126
2506 0.0014025259
2507 0.00082462054
2508 0.0032293969
2509 0.0014926322
2510 0.00096258713
2511 0.00070203136
2512 0.004756453
2513 0.0002456984
2514 0.0015310615
2515 0.0046824375
2516 0.0001895222
2517 0.002596023
2518 0.00040163167
2519 0.00048499252
2520 0.00060386595
2521 0.00032896295
2522 0.00018644294
2523 0.001012353
2524 0.0016944292
2525 0.0022063553
2526 0.0008695367
2

...

7450 0.000276874
7451 0.00016206055
7452 0.00046905104
7453 0.00044821517
7454 0.0006961027
7455 0.00086127454
7456 0.00013262729
7457 0.0011055806
7458 0.00057265407
7459 3.1133164e-05
7460 0.0010600482
7461 0.00023334571
7462 0.00029928001
7463 0.0009675718
7464 1.2497718e-05
7465 0.00074893615
7466 0.00034690695
7467 0.00070632086
7468 0.00022282868
7469 0.0002148929
7470 0.00019106634
7471 6.0013153e-05
7472 0.0012697636
7473 0.00027821254
7474 1.0723914e-05
7475 0.00015264528
7476 4.5888803e-05
7477 4.6920923e-05
7478 0.0009359303
7479 0.00018854043
7480 0.00029757284
7481 0.00016776125
7482 0.00037916354
7483 0.0010090634
7484 3.469224e-05
7485 0.00053001504
7486 7.738179e-05
7487 0.00042904695
7488 5.379271e-07
7489 0.0006514878
7490 0.000388348
7491 0.0013173397
7492 0.00019490335
7493 1.4260636e-06
7494 3.5171272e-06
7495 0.00063428364
7496 0.00018735394
7497 0.00053599826
7498 0.00018466807
7499 0.00013460743
7500 0.00032584608
7501 0.0002529944
7502 7.961876e-05
7503 0.00128

8096 2.7397908e-05
8097 3.172998e-05
8098 0.00029185295
8099 7.959209e-06
8100 0.00043522732
8101 0.00020941313
8102 0.00073823816
8103 0.00066317205
8104 0.00016059336
8105 0.00031155857
8106 0.0007962808
8107 8.031273e-05
8108 3.377253e-06
8109 0.000155348
8110 0.00014315211
8111 0.0010728131
8112 3.9213705e-06
8113 0.0005301907
8114 6.9951875e-07
8115 0.00076366897
8116 0.00064675586
8117 0.00036609115
8118 0.00066600554
8119 0.0008298973
8120 2.6247159e-05
8121 8.386815e-05
8122 1.2136669e-05
8123 0.00023535675
8124 2.5318055e-05
8125 0.00014795846
8126 4.4647248e-05
8127 0.00026459014
8128 0.0008873011
8129 5.593027e-05
8130 8.9303976e-05
8131 1.0360481e-05
8132 0.0005603742
8133 0.00080693944
8134 0.0008377664
8135 0.0007139131
8136 0.00012540018
8137 6.224529e-05
8138 3.6979105e-05
8139 0.0006026011
8140 6.425228e-05
8141 0.0005779724
8142 0.00058646064
8143 0.00030540093
8144 7.061848e-05
8145 1.224155e-09
8146 0.0003016874
8147 0.00017861447
8148 6.1473664e-05
8149 0.000586330

8732 0.0009117019
8733 1.5700594e-05
8734 2.9500901e-05
8735 9.868789e-06
8736 5.171909e-05
8737 2.4649522e-05
8738 0.0008152811
8739 2.8358897e-06
8740 3.81871e-05
8741 0.00027659832
8742 0.000850081
8743 1.3706899e-05
8744 4.2459546e-05
8745 0.00057456986
8746 0.00044153957
8747 0.0006018522
8748 6.674612e-05
8749 0.0002559095
8750 0.00031508453
8751 8.626719e-05
8752 8.527247e-05
8753 0.000575948
8754 4.9771967e-05
8755 1.1451013e-05
8756 0.00023008845
8757 1.6394426e-05
8758 0.00031118834
8759 3.529658e-08
8760 0.0002910633
8761 0.00020033789
8762 8.256646e-05
8763 0.0006192157
8764 0.00026895932
8765 0.00026449707
8766 0.0005236207
8767 7.477928e-05
8768 0.0007629971
8769 8.243761e-05
8770 0.00065990665
8771 0.0006594749
8772 3.974149e-05
8773 0.0003994439
8774 4.591869e-05
8775 0.00020005283
8776 0.00096188457
8777 3.8190046e-07
8778 2.4291576e-05
8779 5.665294e-05
8780 0.0002519015
8781 0.00024188646
8782 4.7005888e-05
8783 0.00042007669
8784 0.00014850273
8785 0.0009301115
8786

9373 1.997922e-05
9374 1.1859604e-05
9375 0.00058231666
9376 0.00030442883
9377 0.0007762917
9378 0.00017923157
9379 6.692154e-05
9380 0.00047271655
9381 0.0006539883
9382 0.00011138599
9383 0.0006206998
9384 2.9737706e-05
9385 0.0002177332
9386 0.00021090973
9387 0.00043326733
9388 5.5746004e-06
9389 0.0005463783
9390 4.8604838e-05
9391 1.764285e-07
9392 0.00023290704
9393 0.0003663375
9394 0.00011269196
9395 0.00022173562
9396 0.00043770784
9397 0.00078153843
9398 0.00082943717
9399 0.00010745566
9400 1.675453e-05
9401 0.00077389885
9402 0.00032928734
9403 5.5664023e-05
9404 8.367902e-06
9405 1.3974322e-05
9406 3.4324646e-06
9407 7.2766336e-05
9408 0.00010749891
9409 0.00015662095
9410 6.2073304e-05
9411 0.0002445338
9412 9.413511e-05
9413 1.5262762e-06
9414 2.0905129e-05
9415 0.00010895485
9416 0.00018461623
9417 0.0006199605
9418 7.122283e-05
9419 0.0004970005
9420 0.0005843292
9421 3.859556e-05
9422 0.0001993958
9423 5.133629e-06
9424 0.00026289807
9425 0.00054066983
9426 0.000692

Learned parameters:  m=0.0998, b=0.5368


Often times if we are developing a model, we want to inspect the graph and its variables and performance. We can (and usually will) just print or plot the values for quick visualization. However, Tensorflow has greater tools for inspecting our graph and model, such as Tensorboard. Below we will run the same toy linear regression model but save the variable data we are interested in. 

In [31]:
import datetime

tf.reset_default_graph()

### build the graph
## first set up the parameters
m = tf.get_variable("m", [], initializer=tf.constant_initializer(0.))
## save our variable for Tensorboard
tf.summary.histogram("m_prediction",m)
b = tf.get_variable("b", [], initializer=tf.constant_initializer(0.))
tf.summary.histogram("b_prediction",b)

init = tf.global_variables_initializer()

## then set up the computations
input_placeholder = tf.placeholder(tf.float32, name="input")
output_placeholder = tf.placeholder(tf.float32, name="output")

x = input_placeholder
y = output_placeholder
tf.summary.histogram("y_true",output_placeholder)
y_guess = m * x + b
tf.summary.histogram("y_guess",y_guess)

loss = tf.square(y - y_guess)
tf.summary.scalar("loss", loss)

## finally, set up the optimizer and minimization node
optimizer = tf.train.GradientDescentOptimizer(1e-3)
train_op = optimizer.minimize(loss)

### start the session
sess = tf.Session()
## merge all our Tensorboard summaries and create file directory
summaryMerged = tf.summary.merge_all()
filename="./summary_log/run"+datetime.datetime.now().strftime("%Y-%m-%d--%H-%M-%s")
writer = tf.summary.FileWriter(filename, sess.graph)

## run variables initializer
sess.run(init)

### perform the training loop
import random

## set up problem
true_m = random.random()
true_b = random.random()

for update_i in range(20000):
    ## (1) get the input and output
    input_data = random.random()
    output_data = true_m * input_data + true_b
    
    ## (2), (3), and (4) all take place within a single call to sess.run()!
    _loss, _, sumOut = sess.run([loss, train_op,summaryMerged], feed_dict={input_placeholder: input_data, output_placeholder: output_data})
    print(update_i, _loss)
    
    if update_i % 100 == 0:
        ## write all of the variables to file
        writer.add_summary(sumOut, update_i)

### finally, print out the values we learned for our two variables
print("True parameters:     m=%.4f, b=%.4f" % (true_m, true_b))
print("Learned parameters:  m=%.4f, b=%.4f" % tuple(sess.run([m, b])))

0 0.9116887
1 0.7013267
2 1.5542613
3 2.2132394
4 1.5760517
5 2.4288127
6 1.4219137
7 1.2820653
8 2.0597405
9 1.8959999
10 1.2018756
11 1.5399643
12 1.6719115
13 0.5497208
14 0.5287568
15 0.94567364
16 0.9564808
17 0.8646594
18 1.0101318
19 1.078123
20 0.9258942
21 0.95276743
22 1.7634654
23 1.0898392
24 1.5794253
25 0.5222318
26 0.7786576
27 1.9695281
28 1.28069
29 0.5510149
30 2.101739
31 1.0032141
32 1.1897514
33 2.0629613
34 0.983484
35 0.6559144
36 2.0092714
37 1.0073972
38 0.58854276
39 0.5567753
40 1.4315878
41 1.1525848
42 1.1391612
43 0.92881525
44 1.0560946
45 1.7304327
46 0.47216538
47 0.91965276
48 1.132173
49 1.7276748
50 1.3261093
51 1.3712416
52 0.76626486
53 1.4227372
54 1.4958533
55 1.3060111
56 0.4135065
57 1.8655859
58 1.3154675
59 0.42588043
60 0.8167687
61 0.603556
62 0.65825224
63 1.5256054
64 0.41930285
65 1.471571
66 1.7467767
67 0.6049075
68 0.6788493
69 1.0573981
70 0.84772944
71 0.97173977
72 0.7089271
73 0.64775413
74 0.4828854
75 0.8259549
76 1.3386271
77 0

569 0.17873439
570 0.16434474
571 0.044148646
572 0.12664966
573 0.052340902
574 0.11796176
575 0.031248635
576 0.076513566
577 0.10267098
578 0.11193648
579 0.06935251
580 0.03623866
581 0.008638869
582 0.026005175
583 0.107525714
584 0.1424981
585 0.006104326
586 0.18689822
587 0.08352557
588 0.109623805
589 0.16435847
590 0.046302553
591 0.06494659
592 0.070101626
593 0.03993143
594 0.01375646
595 0.174556
596 0.18098547
597 0.030608337
598 0.10020036
599 0.04839482
600 0.004151816
601 0.08423271
602 0.14586113
603 0.18961033
604 0.13497132
605 0.12107881
606 0.15074602
607 0.07085045
608 0.014306341
609 0.07468165
610 0.024931084
611 0.10124919
612 0.12634337
613 0.08042137
614 0.16023591
615 0.14790528
616 0.07441052
617 0.001614607
618 0.09500671
619 0.021350361
620 0.17728607
621 0.01062569
622 0.18430531
623 0.016873587
624 0.12784193
625 0.011379091
626 0.112647854
627 0.067836754
628 0.020170033
629 0.002203109
630 0.009145391
631 0.017188085
632 0.004177665
633 0.15926516
63

1114 0.043018356
1115 0.03554683
1116 0.00012401969
1117 0.045907494
1118 0.020212973
1119 0.012087025
1120 0.0031588986
1121 0.02934236
1122 0.0015393615
1123 0.00922247
1124 0.0043098424
1125 0.0053238687
1126 0.0008234707
1127 1.6160204e-05
1128 0.01960033
1129 0.0002528901
1130 0.018059453
1131 0.0053467606
1132 0.003484245
1133 0.03087039
1134 0.041126937
1135 0.02005654
1136 0.0042119687
1137 0.0019176357
1138 0.001038728
1139 0.0064150267
1140 0.0031538622
1141 0.004778059
1142 0.010330541
1143 0.040136177
1144 0.005372489
1145 0.0053581684
1146 0.0019106364
1147 0.025855867
1148 0.008717583
1149 0.00023511359
1150 0.02624035
1151 0.009727727
1152 0.0066367416
1153 0.014937797
1154 0.0010279447
1155 0.041196447
1156 0.0027677536
1157 0.002459463
1158 0.034121096
1159 0.007697461
1160 0.008697304
1161 0.017045114
1162 0.0015946227
1163 0.03881225
1164 0.00820806
1165 0.00028278655
1166 0.0025503198
1167 0.024308639
1168 0.0015019408
1169 0.0032230187
1170 0.00046680757
1171 0.003

1683 0.0104347
1684 0.018445851
1685 0.013557232
1686 0.009853678
1687 0.0015662924
1688 0.00868899
1689 0.0007470875
1690 0.009655645
1691 0.0069508404
1692 0.0068400316
1693 0.0021938249
1694 0.010933122
1695 0.00040192797
1696 0.0005624304
1697 0.00071984023
1698 0.00024019301
1699 0.0022245406
1700 0.008449706
1701 0.012140556
1702 0.0003306299
1703 0.00053161354
1704 0.019220268
1705 0.0009090978
1706 0.0030884363
1707 0.015051248
1708 0.0014781551
1709 0.00033248364
1710 0.0070193363
1711 0.012403774
1712 0.0002882938
1713 0.011296864
1714 0.01792868
1715 4.4264933e-05
1716 0.006594157
1717 0.009922028
1718 0.00240852
1719 0.0005249008
1720 0.00801958
1721 0.0016423496
1722 0.002558984
1723 6.44006e-06
1724 0.0012462427
1725 0.010824439
1726 1.5810372e-05
1727 0.016004575
1728 0.0155866975
1729 0.009915474
1730 0.006788811
1731 0.009140957
1732 0.006590227
1733 0.0044451514
1734 0.00012248186
1735 0.0006576364
1736 0.015476908
1737 0.0020470608
1738 0.0010310583
1739 0.0002172216

2228 0.008821212
2229 0.014863874
2230 0.0012459649
2231 0.0013955613
2232 0.010222373
2233 0.01297184
2234 0.007884637
2235 0.0011483226
2236 0.007163914
2237 0.0117304325
2238 0.008710805
2239 0.014110746
2240 0.006303442
2241 0.0054112137
2242 0.003905341
2243 0.0030827548
2244 0.0022450085
2245 7.37919e-05
2246 0.009562383
2247 2.7821969e-05
2248 0.003364494
2249 0.0004933583
2250 0.010586011
2251 0.005164196
2252 0.010861343
2253 0.008085057
2254 0.0051542292
2255 0.008808978
2256 0.0008978038
2257 0.011781979
2258 0.0032411544
2259 0.007511913
2260 0.007904752
2261 0.001014968
2262 0.008296296
2263 3.9832253e-06
2264 0.0052627153
2265 0.008090235
2266 0.009275907
2267 0.0009432934
2268 1.6526463e-05
2269 0.0058114748
2270 0.0005239508
2271 0.0056643253
2272 0.0017587114
2273 0.007441151
2274 0.009347065
2275 0.00073134236
2276 0.0014715078
2277 0.008663917
2278 0.0013211067
2279 0.003697548
2280 0.01303633
2281 0.009542996
2282 4.477812e-06
2283 0.0025708112
2284 3.294129e-05
228

...

17575 4.6333213e-07
17576 3.5431085e-06
17577 8.114665e-06
17578 3.4190055e-06
17579 2.1776664e-06
17580 0.00018043854
17581 4.2462652e-05
17582 1.1530619e-05
17583 0.00011546375
17584 0.00021219626
17585 1.6012539e-06
17586 4.110189e-07
17587 2.9993826e-06
17588 1.301758e-05
17589 2.9311494e-05
17590 2.0549645e-05
17591 4.7092562e-05
17592 6.839341e-06
17593 0.00018792549
17594 9.681175e-05
17595 6.733958e-05
17596 0.00023751789
17597 2.0222398e-05
17598 6.707083e-07
17599 3.1367683e-06
17600 1.98797e-05
17601 7.7471996e-05
17602 0.0001780223
17603 0.00011547016
17604 2.3262184e-05
17605 3.0208923e-08
17606 0.0002586553
17607 0.00014273166
17608 0.0002667391
17609 1.5183446e-05
17610 6.527494e-05
17611 2.17837e-06
17612 6.4263746e-05
17613 0.00013548572
17614 0.00014528086
17615 3.9208968e-05
17616 2.2300487e-06
17617 0.00011291859
17618 1.5587435e-05
17619 1.2506569e-05
17620 1.911883e-06
17621 0.00021095993
17622 3.3325556e-05
17623 0.00011040681
17624 0.0002817871
17625 3.1883937e-

18121 4.5202914e-06
18122 2.968905e-06
18123 1.226455e-06
18124 0.00017157014
18125 0.00011017645
18126 7.5366996e-05
18127 5.3972097e-05
18128 2.7901253e-05
18129 0.00011750428
18130 0.000101054
18131 0.00018303069
18132 9.395626e-06
18133 0.00025167453
18134 2.986184e-06
18135 5.6333072e-05
18136 8.406696e-05
18137 3.5829045e-05
18138 1.3093995e-06
18139 8.222668e-05
18140 4.3596494e-05
18141 2.7563563e-06
18142 0.00010567622
18143 7.8205056e-05
18144 3.6409358e-05
18145 1.4764599e-05
18146 0.00024271407
18147 4.6432797e-06
18148 6.0136965e-05
18149 1.9676675e-06
18150 0.00019484176
18151 0.00017545336
18152 2.7472212e-05
18153 3.8855946e-05
18154 0.00020298721
18155 8.434153e-05
18156 5.7895217e-05
18157 6.179936e-05
18158 9.887897e-06
18159 0.00023090288
18160 3.2333816e-09
18161 8.6733955e-05
18162 0.00015857842
18163 6.729361e-05
18164 0.00018444458
18165 5.025891e-06
18166 3.701922e-08
18167 1.2455189e-05
18168 0.00010259613
18169 3.6641344e-05
18170 0.0001001608
18171 0.0001328

18673 2.7424745e-05
18674 9.086764e-05
18675 9.434573e-05
18676 0.00013968606
18677 0.00021302018
18678 4.182652e-05
18679 3.326915e-05
18680 1.1229021e-05
18681 7.464121e-05
18682 1.0911721e-05
18683 2.1844768e-05
18684 7.502793e-05
18685 4.7986816e-07
18686 2.4436915e-05
18687 7.7631565e-05
18688 0.00011086071
18689 0.00019224136
18690 2.6311323e-05
18691 0.00017616621
18692 0.0002036178
18693 0.00015169532
18694 1.7901159e-05
18695 4.260053e-06
18696 2.04841e-06
18697 2.6491405e-05
18698 0.00010441046
18699 0.00017033413
18700 7.950605e-05
18701 0.00014698709
18702 6.519598e-05
18703 2.2491336e-05
18704 9.163223e-06
18705 1.7787352e-05
18706 0.00015563935
18707 0.0001880219
18708 8.017835e-06
18709 3.404649e-05
18710 0.00016579531
18711 2.0226687e-05
18712 0.00013645181
18713 1.598624e-05
18714 0.00011614623
18715 0.0001271322
18716 3.983699e-05
18717 5.4216718e-05
18718 0.00017753434
18719 8.633141e-05
18720 0.0001455166
18721 2.0600282e-06
18722 0.00011477563
18723 4.0221643e-05
1

19165 9.163223e-06
19166 3.3727276e-05
19167 8.545392e-06
19168 9.153248e-05
19169 3.100293e-05
19170 3.5109155e-05
19171 0.00017863838
19172 9.1906946e-07
19173 0.00011676114
19174 0.00019364222
19175 6.6533976e-05
19176 7.520157e-06
19177 0.00013689637
19178 0.00019626207
19179 0.00012039052
19180 3.0281028e-05
19181 7.122585e-05
19182 5.1983857e-06
19183 2.2188206e-05
19184 5.2470223e-05
19185 4.4774784e-05
19186 7.0283068e-06
19187 0.00010592391
19188 7.712613e-07
19189 4.498521e-06
19190 2.3383081e-05
19191 1.7042282e-06
19192 9.0583766e-05
19193 0.00019754341
19194 9.050777e-05
19195 8.125023e-05
19196 7.0479146e-06
19197 1.1389377e-05
19198 4.9660175e-05
19199 6.234597e-05
19200 0.000116712195
19201 1.1342756e-05
19202 2.226464e-05
19203 0.00017239088
19204 3.7464204e-05
19205 3.1173087e-05
19206 7.675388e-05
19207 3.5355395e-05
19208 0.00013141235
19209 4.777977e-05
19210 5.567285e-06
19211 6.368784e-05
19212 7.46443e-05
19213 1.055684e-06
19214 2.6573687e-05
19215 8.052012e-05

19691 5.0784598e-05
19692 1.1865763e-05
19693 1.5296997e-05
19694 0.00015460598
19695 6.23648e-05
19696 5.9062062e-05
19697 2.0611296e-05
19698 1.543157e-05
19699 6.0720777e-05
19700 0.00016852331
19701 5.5993964e-06
19702 4.336382e-05
19703 1.5255064e-05
19704 1.1029384e-05
19705 6.7231034e-05
19706 8.1672e-05
19707 7.003166e-09
19708 2.6218727e-06
19709 3.0925876e-06
19710 8.289825e-05
19711 1.6184174e-05
19712 4.8018182e-06
19713 1.3008979e-05
19714 0.0001461702
19715 1.7113944e-06
19716 2.4041343e-05
19717 3.1231684e-05
19718 1.0823062e-06
19719 0.00012511201
19720 9.2304645e-06
19721 9.0105605e-05
19722 3.928692e-06
19723 8.031273e-05
19724 9.455195e-05
19725 3.6084803e-06
19726 2.6600734e-05
19727 6.159148e-05
19728 9.832839e-05
19729 6.565913e-06
19730 7.2751084e-07
19731 4.5695942e-06
19732 0.00015344756
19733 1.5025922e-05
19734 9.595507e-05
19735 8.409429e-05
19736 0.0001031499
19737 1.8820538e-05
19738 6.0535134e-05
19739 2.1534574e-06
19740 7.6093386e-08
19741 7.2284115e-07

Now, open up your terminal console and cd to the same file directory location our program is in

Then run ```tensorboard --logdir=./summary_log/``` and copy the location http address into a new browser window. You should see our graph, scalars, histogram and even more. Run the model more than once, and you can compare the results between runs. Very useful!

For more details see the source I adapted this from: https://thecodacus.com/tensorboard-tutorial-visualize-networks-graphically/

We can also just visualize our graph inline with Jupyter via [the following code](https://stackoverflow.com/questions/38189119/simple-way-to-visualize-a-tensorflow-graph-in-jupyter):

In [32]:
import numpy as np
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [33]:
show_graph(tf.get_default_graph().as_graph_def())

You can zoom and click each node and operation to see its inputs and outputs, which can be very helpful for debugging or just understanding what is going on.

Furthermore and importantly, we can save and reload trained models. This is essential for any serious machine/deep learning project that takes significant time to train. Now, we can train something for weeks or even months, or even forever.

In [34]:
tf.reset_default_graph()

### build the graph
## first set up the parameters
m = tf.get_variable("m", [], initializer=tf.constant_initializer(0.))
## save our variable for Tensorboard
tf.summary.histogram("m_prediction",m)
b = tf.get_variable("b", [], initializer=tf.constant_initializer(0.))
tf.summary.histogram("b_prediction",b)

init = tf.global_variables_initializer()

## then set up the computations
input_placeholder = tf.placeholder(tf.float32)
output_placeholder = tf.placeholder(tf.float32)

x = input_placeholder
y = output_placeholder
tf.summary.histogram("y_true",output_placeholder)
y_guess = m * x + b
tf.summary.histogram("y_guess",y_guess)

loss = tf.square(y - y_guess)
tf.summary.scalar("loss", loss)

## finally, set up the optimizer and minimization node
optimizer = tf.train.GradientDescentOptimizer(1e-3)
train_op = optimizer.minimize(loss)

### start the session
sess = tf.Session()
## merge all our Tensorboard summaries and create file directory
summaryMerged = tf.summary.merge_all()
filename="./summary_log/run"+datetime.datetime.now().strftime("%Y-%m-%d--%H-%M-%s")
writer = tf.summary.FileWriter(filename, sess.graph)

## create tf.saver
saver = tf.train.Saver()

## run variables initializer
sess.run(init)

### perform the training loop
import random

## set up problem
true_m = random.random()
true_b = random.random()

for update_i in range(20000):
    ## (1) get the input and output
    input_data = random.random()
    output_data = true_m * input_data + true_b
    
    ## (2), (3), and (4) all take place within a single call to sess.run()!
    _loss, _, sumOut = sess.run([loss, train_op,summaryMerged], feed_dict={input_placeholder: input_data, output_placeholder: output_data})
    print(update_i, _loss)
    
    if update_i % 100 == 0:
        ## write all of the variables to file
        writer.add_summary(sumOut, update_i)

## save to directory
save_path = saver.save(sess, './model/weights')

### finally, print out the values we learned for our two variables
print("True parameters:     m=%.4f, b=%.4f" % (true_m, true_b))
print("Learned parameters:  m=%.4f, b=%.4f" % tuple(sess.run([m, b])))
print("Model saved in path: %s" % save_path)

0 0.011425321
1 0.24552952
2 0.2486584
3 0.042790644
4 0.20095299
5 0.07214198
6 0.24520616
7 0.14315243
8 0.03423859
9 0.08776448
10 0.08974413
11 0.11765186
12 0.25790864
13 0.09369954
14 0.19591361
15 0.04792686
16 0.07377299
17 0.07515374
18 0.011649444
19 0.27031586
20 0.088169135
21 0.13645378
22 0.24190982
23 0.034058545
24 0.108238
25 0.09552109
26 0.021768484
27 0.13513188
28 0.13223696
29 0.2436539
30 0.1065539
31 0.2487551
32 0.085850544
33 0.076526605
34 0.23983166
35 0.016687388
36 0.008581595
37 0.1441896
38 0.011955626
39 0.14871825
40 0.20482205
41 0.20492244
42 0.1328469
43 0.018957134
44 0.00536841
45 0.04476177
46 0.24253072
47 0.010598653
48 0.09848992
49 0.018905636
50 0.031337894
51 0.06367727
52 0.040823083
53 0.07711339
54 0.162828
55 0.23387739
56 0.082447655
57 0.14066839
58 0.03415751
59 0.018254729
60 0.037948806
61 0.057238013
62 0.033727705
63 0.025053017
64 0.04032782
65 0.08611094
66 0.14330229
67 0.05467249
68 0.010052117
69 0.06383344
70 0.0046844054
7

576 0.0035376821
577 0.023774356
578 0.0039202548
579 0.009187563
580 0.008655193
581 0.011838067
582 0.028729783
583 0.0024997867
584 0.0095652975
585 0.000101875936
586 0.008165202
587 0.00047470466
588 0.002353479
589 0.0074848803
590 0.040331353
591 0.0044820863
592 0.034340065
593 0.007538914
594 0.0019239653
595 0.026790973
596 0.0031510645
597 0.0023451846
598 0.00030942177
599 0.006355928
600 0.00078989053
601 0.005361389
602 0.011202124
603 0.036845222
604 0.002658177
605 0.0070175137
606 0.02257282
607 0.015194694
608 0.001959021
609 0.0014767622
610 0.0036736908
611 0.0038657906
612 0.033871647
613 0.0036382962
614 0.0026337837
615 0.0170616
616 0.03651316
617 0.00083518575
618 0.0015243008
619 0.03837439
620 0.00081189617
621 0.022063358
622 0.0037014724
623 0.0018197432
624 0.0068842224
625 0.0042864275
626 0.002216509
627 0.00020691364
628 0.02836754
629 0.03042088
630 0.011980688
631 0.024104862
632 0.0023491275
633 0.0039282152
634 0.020970047
635 5.9858844e-06
636 0.04

1138 0.006859769
1139 0.0013346883
1140 0.005658587
1141 0.015511008
1142 0.0010686899
1143 0.00305239
1144 0.00016867346
1145 0.00744329
1146 0.00636113
1147 0.0013815707
1148 0.000929937
1149 0.0019987265
1150 0.007681624
1151 0.0013522577
1152 0.011150743
1153 0.00013432336
1154 0.0064864904
1155 0.0017831082
1156 0.012868063
1157 0.010882847
1158 0.0022636016
1159 0.0005583766
1160 0.009121666
1161 0.00040654416
1162 0.005163198
1163 0.01844904
1164 0.00027790343
1165 0.0011956502
1166 0.007269924
1167 0.014320197
1168 0.0019366976
1169 0.007510965
1170 0.0009513838
1171 0.011369842
1172 8.359545e-05
1173 0.0021837414
1174 0.0034510656
1175 0.008866747
1176 0.014427881
1177 0.0024319638
1178 0.007161291
1179 0.0072283684
1180 0.018881783
1181 0.0023835571
1182 0.019202119
1183 0.004109119
1184 3.0915045e-05
1185 0.0015757541
1186 0.0094876615
1187 4.7275986e-05
1188 0.007251437
1189 0.004929962
1190 0.00032432217
1191 0.002659271
1192 0.0016389685
1193 0.013375832
1194 0.0005984033

1661 0.0036262693
1662 0.009743538
1663 0.0017210556
1664 6.781044e-05
1665 0.004995836
1666 0.0027467995
1667 0.0096033905
1668 0.0027404835
1669 0.005401206
1670 0.014605519
1671 0.016337438
1672 0.0011263521
1673 0.014050048
1674 3.2120606e-06
1675 0.00027075215
1676 0.016655535
1677 0.0052267257
1678 0.0054390915
1679 2.6747362e-06
1680 0.014151203
1681 0.00033689442
1682 0.00029399627
1683 0.0009089486
1684 0.004573449
1685 0.0015584351
1686 0.0040063146
1687 0.009701747
1688 0.00018754898
1689 0.006732324
1690 0.0037273897
1691 0.012284625
1692 0.00302655
1693 0.01650636
1694 8.337755e-06
1695 0.0036311452
1696 0.0005560171
1697 0.0002651586
1698 0.0035239668
1699 7.356372e-05
1700 0.010094941
1701 0.00013191505
1702 0.01655826
1703 0.0014122865
1704 0.0007085562
1705 0.0014714941
1706 0.001812613
1707 0.0016952193
1708 0.0003533926
1709 0.004597125
1710 0.0026406494
1711 0.0017859447
1712 0.0080300765
1713 0.0009177879
1714 0.0018901173
1715 8.378193e-05
1716 0.0053761546
1717 0

2230 0.0069508553
2231 0.0069792243
2232 0.004085009
2233 0.004908431
2234 0.0015264051
2235 0.00081529305
2236 0.012902912
2237 0.007895692
2238 0.0044014384
2239 0.0011957862
2240 0.00427513
2241 0.004056529
2242 0.006180041
2243 0.0026007658
2244 0.008879474
2245 0.00070240884
2246 0.00081023265
2247 0.0034870214
2248 0.007913199
2249 0.00062322384
2250 0.004979693
2251 0.0038347011
2252 0.0025003678
2253 0.00085351494
2254 0.0006892208
2255 0.008858756
2256 0.0050323256
2257 8.48975e-05
2258 0.0017230468
2259 0.0015741055
2260 0.001422798
2261 0.0012242885
2262 0.008584199
2263 0.011159544
2264 0.0055232923
2265 0.009948965
2266 5.6964273e-05
2267 0.0041254465
2268 0.013236877
2269 1.406486e-06
2270 0.0025627424
2271 0.0058401497
2272 4.1911944e-06
2273 0.0033811827
2274 0.0036506527
2275 0.0131519195
2276 0.0057857395
2277 0.0039060954
2278 0.004003014
2279 0.00039371318
2280 0.00028868663
2281 0.0060799713
2282 0.0029870437
2283 0.0077437586
2284 0.0006022851
2285 0.00019034528
2

...

17601 1.2313394e-05
17602 3.080346e-07
17603 2.322762e-06
17604 3.3873513e-07
17605 5.7439414e-05
17606 8.032525e-06
17607 6.904232e-05
17608 2.80368e-07
17609 6.849009e-06
17610 7.1658105e-05
17611 0.00012978134
17612 6.344218e-06
17613 1.1899047e-05
17614 8.70961e-06
17615 0.00018638597
17616 0.00023256696
17617 0.00011240361
17618 0.00017175991
17619 2.0869988e-05
17620 1.097302e-06
17621 8.629231e-06
17622 2.5916577e-05
17623 7.616806e-05
17624 0.00011696507
17625 3.3994158e-06
17626 7.730893e-05
17627 2.4928479e-05
17628 0.00015030224
17629 0.000107941545
17630 0.00013231892
17631 7.63531e-05
17632 4.5777026e-05
17633 0.00010565906
17634 2.5057801e-05
17635 0.0001196083
17636 3.620249e-05
17637 0.00010472161
17638 1.8952127e-05
17639 9.833431e-06
17640 0.00013480597
17641 3.334758e-05
17642 0.00014559571
17643 0.00015040858
17644 3.798001e-05
17645 3.7857422e-05
17646 0.0001786814
17647 7.95103e-09
17648 3.584968e-06
17649 1.0618171e-06
17650 0.00010807194
17651 1.457853e-05
17652

18157 2.9945442e-05
18158 7.5743574e-06
18159 4.1553463e-05
18160 5.074128e-05
18161 0.00015177536
18162 0.00012289184
18163 6.359891e-05
18164 1.8582112e-05
18165 9.985975e-06
18166 7.284822e-05
18167 8.05121e-05
18168 0.00013627918
18169 8.7462664e-05
18170 4.9455057e-06
18171 3.2647233e-05
18172 4.480231e-05
18173 2.3130706e-05
18174 0.00016557513
18175 0.00011812828
18176 2.129732e-06
18177 3.4653633e-05
18178 0.00013114738
18179 1.4330476e-06
18180 4.0657345e-05
18181 9.267023e-05
18182 8.7551874e-05
18183 3.0312853e-05
18184 3.2348545e-08
18185 1.8571835e-05
18186 4.288519e-06
18187 2.5586283e-05
18188 8.218183e-05
18189 4.6138673e-05
18190 7.80628e-05
18191 6.232244e-05
18192 6.9923466e-05
18193 4.1662846e-05
18194 5.0155784e-05
18195 3.2729158e-06
18196 1.145331e-08
18197 4.555147e-05
18198 9.308324e-05
18199 5.6632758e-05
18200 8.714712e-06
18201 1.1629805e-05
18202 0.00011503759
18203 9.274196e-05
18204 6.3910644e-05
18205 6.451561e-06
18206 0.00021181723
18207 0.00010887399


18730 0.00012500536
18731 3.817642e-05
18732 5.7115976e-05
18733 0.000117481024
18734 9.690326e-05
18735 3.9109378e-05
18736 0.00017023535
18737 1.2158505e-06
18738 3.5258e-05
18739 5.21e-05
18740 1.611854e-05
18741 9.962376e-05
18742 0.00014755128
18743 9.862768e-05
18744 1.9534136e-05
18745 8.436616e-05
18746 1.4178493e-05
18747 0.00012483716
18748 2.727668e-05
18749 0.000111398884
18750 3.6123634e-05
18751 5.516353e-05
18752 0.00013839426
18753 8.282338e-07
18754 9.619578e-07
18755 0.00014102951
18756 0.00016724401
18757 8.339175e-05
18758 0.00020549989
18759 8.0946425e-05
18760 2.387097e-05
18761 6.167595e-05
18762 4.2414893e-05
18763 0.00010634884
18764 5.9475737e-05
18765 8.970999e-05
18766 9.3174116e-05
18767 0.00011587884
18768 6.651915e-05
18769 2.2925624e-05
18770 3.4389224e-05
18771 4.011738e-05
18772 0.00015275393
18773 0.00011740222
18774 5.8245412e-05
18775 6.2859544e-05
18776 3.267159e-05
18777 0.00011253889
18778 0.00020378498
18779 0.00014805564
18780 8.304572e-06
1878

19313 2.0833517e-05
19314 6.981485e-05
19315 3.0553845e-06
19316 5.079352e-05
19317 0.00013553533
19318 9.448473e-05
19319 3.4687677e-05
19320 0.00010607119
19321 1.1939793e-05
19322 4.0938503e-05
19323 7.786284e-06
19324 4.1435014e-05
19325 3.9601805e-06
19326 4.8737904e-05
19327 1.29482105e-05
19328 0.00012256367
19329 4.475544e-05
19330 6.528649e-07
19331 7.406147e-05
19332 9.091537e-07
19333 1.1858706e-06
19334 6.02535e-07
19335 1.356647e-07
19336 0.00010693112
19337 0.00016812391
19338 0.0001436296
19339 8.4896405e-05
19340 3.8519316e-05
19341 8.342305e-05
19342 0.000103813225
19343 6.0262755e-05
19344 2.1292972e-06
19345 2.056095e-06
19346 1.7202951e-06
19347 0.00011997755
19348 4.800904e-06
19349 8.898207e-05
19350 1.120967e-06
19351 8.033383e-05
19352 9.393281e-05
19353 1.167766e-06
19354 3.1615928e-05
19355 5.444351e-06
19356 0.000118174605
19357 0.00012918719
19358 9.722242e-05
19359 6.5422605e-06
19360 9.992353e-05
19361 1.0755558e-05
19362 1.5528658e-05
19363 4.0574414e-06


19876 4.0852232e-08
19877 3.3493336e-05
19878 4.4421886e-06
19879 5.7413274e-10
19880 3.7037484e-05
19881 0.00010031954
19882 1.2188637e-05
19883 4.2980802e-05
19884 4.695572e-08
19885 3.9646857e-05
19886 0.00012213051
19887 6.558857e-05
19888 3.4152592e-06
19889 2.6785967e-05
19890 9.800476e-05
19891 7.550159e-07
19892 5.5099357e-05
19893 9.1652844e-05
19894 9.159614e-06
19895 7.793778e-05
19896 0.00011626959
19897 4.281391e-05
19898 4.8757878e-09
19899 4.1377294e-05
19900 9.1006325e-05
19901 0.00010096444
19902 1.0766507e-05
19903 6.717338e-07
19904 7.79391e-05
19905 9.9853234e-05
19906 0.00012424514
19907 6.654467e-05
19908 2.8545952e-05
19909 4.1014046e-05
19910 5.9029084e-05
19911 7.1120754e-05
19912 0.00011194782
19913 0.00016499082
19914 8.458475e-09
19915 2.9523891e-05
19916 5.829182e-05
19917 5.431841e-06
19918 6.847788e-05
19919 8.612441e-05
19920 3.1090443e-05
19921 7.437677e-08
19922 4.3416825e-05
19923 2.30694e-05
19924 0.00010788427
19925 8.207636e-08
19926 1.2535464e-05


So now we should have a folder "model" with our weight files saved (these are all of our variables). We can now reload the model and continue working with it, without having to retrain from scratch. More on working with and saving models here: https://www.tensorflow.org/guide/saved_model

In [35]:
sess = tf.Session()

with tf.Session() as sess:
    # Reset all variables to initial state
    sess.run(init)
    
    # Verify reset variable
    print('reset value of var m',m.eval())
    
    reader = tf.train.Saver()
    reader.restore(sess, './model/weights')
    
    # Verify restored variable
    print('restored value of var m',m.eval())

reset value of var m 0.0
INFO:tensorflow:Restoring parameters from ./model/weights
restored value of var m 0.4256247


One tricky but essential aspect of working with Tensorflow is managing graphs and sessions. Recall that the graph contains all the nodes and operations we build for our learning model. We call a session when we want to evaluate the graph.

Things can get confusing though if we:
1. have more than one graph, 
2. need to run more than one session,
3. want to reset or update graph elements
4. want to access graph elements for input or output

We won't cover all techniques here, but let's get a bit more familiar with graph and session usage before moving on.

We can assign our default graph (which is automatically instantiated by Tensorflow) to a variable if we wish (though normally we wouldn't - this is just to demonstrate. Normally, you can name a graph if you want at the outset or when constructing a 2nd, different graph).

In [36]:
graph = tf.get_default_graph()

In [37]:
# Verify it's the graph we want
show_graph(graph)

Another way to access all of the nodes in our graph is to call them via graph_def

In [38]:
graphdef = graph.as_graph_def()
for node in graphdef.node:
    print(node)

name: "m/Initializer/Const"
op: "Const"
attr {
  key: "_class"
  value {
    list {
      s: "loc:@m"
    }
  }
}
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 0.0
    }
  }
}

name: "m"
op: "VariableV2"
attr {
  key: "_class"
  value {
    list {
      s: "loc:@m"
    }
  }
}
attr {
  key: "container"
  value {
    s: ""
  }
}
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
    }
  }
}
attr {
  key: "shared_name"
  value {
    s: ""
  }
}

name: "m/Assign"
op: "Assign"
input: "m"
input: "m/Initializer/Const"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "_class"
  value {
    list {
      s: "loc:@m"
    }
  }
}
attr {
  key: "use_locking"
  value {
    b: true
  }
}
attr {
  key: "validate_shape"
  value {
    b: true
  }
}

name: "m/read"
op: "Identity"
input: "m"
attr {
  key: "T"
  valu

We can do similarly access all of the operations in our graph

In [39]:
for op in graph.get_operations(): 
    print(op.name)

m/Initializer/Const
m
m/Assign
m/read
m_prediction/tag
m_prediction
b/Initializer/Const
b
b/Assign
b/read
b_prediction/tag
b_prediction
init
Placeholder
Placeholder_1
y_true/tag
y_true
mul
add
y_guess/tag
y_guess
sub
Square
loss/tags
loss
gradients/Shape
gradients/grad_ys_0
gradients/Fill
gradients/Square_grad/Const
gradients/Square_grad/Mul
gradients/Square_grad/Mul_1
gradients/sub_grad/Shape
gradients/sub_grad/Shape_1
gradients/sub_grad/BroadcastGradientArgs
gradients/sub_grad/Sum
gradients/sub_grad/Reshape
gradients/sub_grad/Sum_1
gradients/sub_grad/Neg
gradients/sub_grad/Reshape_1
gradients/sub_grad/tuple/group_deps
gradients/sub_grad/tuple/control_dependency
gradients/sub_grad/tuple/control_dependency_1
gradients/add_grad/Shape
gradients/add_grad/Shape_1
gradients/add_grad/BroadcastGradientArgs
gradients/add_grad/Sum
gradients/add_grad/Reshape
gradients/add_grad/Sum_1
gradients/add_grad/Reshape_1
gradients/add_grad/tuple/group_deps
gradients/add_grad/tuple/control_dependency
gradi

And get a specific operation by name

In [40]:
some_op = graph.get_operation_by_name("gradients/mul_grad/Shape")
print(some_op)

name: "gradients/mul_grad/Shape"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT32
      tensor_shape {
        dim {
        }
      }
    }
  }
}



Or layer and type

In [41]:
layers = [op.name for op in graph.get_operations() if op.type=='Const' and 'gradients' in op.name]
print(layers)

['gradients/grad_ys_0', 'gradients/Square_grad/Const', 'gradients/add_grad/Shape_1', 'gradients/mul_grad/Shape']


In [47]:
#close and clear our default graph
sess.close()
tf.reset_default_graph()

In [48]:
show_graph(tf.get_default_graph())

In [52]:
train_graph = tf.Graph()
eval_graph = tf.Graph()
infer_graph = tf.Graph()

x = tf.constant(0)

with train_graph.as_default():
    with tf.variable_scope("share", reuse=True):
        a = tf.constant([[-1, -2, -23], [-4, -5, -6]])
        b = tf.constant([[1, 2, 3], [4, 5, 6]])
        c = tf.add(a,b)

with eval_graph.as_default():
    with tf.variable_scope("share_again", reuse=True):
        c1 = c

with infer_graph.as_default():
    with tf.variable_scope("sharing_through_2", reuse=True):
        c2 = c1

In [53]:
show_graph(train_graph)